In [73]:
import os
import json
import cv2
import pandas as pd
from tqdm import tqdm
import numpy as np


In [74]:
os.listdir()

['icon_legend.json',
 '10594-screenshot.jpg',
 'component_legend.json',
 'README.md',
 'measures.csv',
 '.gitignore',
 '10594-hierarchy.json',
 'MakeDataset.ipynb',
 'textButton_legend.json',
 'combined',
 '.git',
 '10594-wireframe.png',
 '10594-metadata.json',
 'model.ipynb',
 'semantic_annotations']

1. Density - Anushka

In [75]:
def calculate_area(bounds):
    return (bounds[2] - bounds[0]) * (bounds[3] - bounds[1])

def do_rectangles_overlap(rect1, rect2):
    return not (rect1[2] <= rect2[0] or rect1[0] >= rect2[2] or rect1[3] <= rect2[1] or rect1[1] >= rect2[3])

def merge_overlapping_objects(data):
    merged_objects = []

    if 'children' in data:
        for parent in data['children']:
            non_overlapping_children = []

            if 'children' in parent:
                for child in parent['children']:
                    overlapping = False
                    for merged_obj in merged_objects:
                        if do_rectangles_overlap(child['bounds'], merged_obj['bounds']):
                            overlapping = True
                            break

                    if not overlapping:
                        non_overlapping_children.append(child)

                merged_objects.extend(non_overlapping_children)

    return merged_objects

def calculate_density_measure(total_area, frame_area):
    if total_area > 0 and frame_area > 0:
        density_measure = 1 - 2 * abs(0.5 - total_area / frame_area)
        density_measure = max(0, min(1, density_measure))
    else:
        density_measure = 0

    return density_measure

def calculate_density(data):
    total_area = 0

    merged_objects = merge_overlapping_objects(data)

    for item in merged_objects:
        bounds = item.get('bounds')
        if bounds:
            area = calculate_area(bounds)
            total_area += area

    frame_bounds = data.get('bounds')
    if frame_bounds:
        frame_area = calculate_area(frame_bounds)

    density_measure = calculate_density_measure(total_area, frame_area)
    scaled_density_measure = 1 - (abs(density_measure - 0.5) * 2)

    return scaled_density_measure

2. Colour - Yash

In [76]:
def calculate_colorfulness(image):
    image = cv2.imread(image)
    # Convert the image to sRGB color space
    srgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Calculate the pixel cloud along directions (rg, yb)
    rg = srgb_image[:,:,0] - srgb_image[:,:,1]
    yb = (srgb_image[:,:,0] + srgb_image[:,:,1]) / 2 - srgb_image[:,:,2]

    # Calculate the standard deviation and mean value along directions (rg, yb)
    std_rg = np.std(rg)
    std_yb = np.std(yb)
    mean_rg = np.mean(rg)
    mean_yb = np.mean(yb)

    # Calculate ^M(3) colorfulness metric
    colorfulness = np.sqrt(std_rg**2 + std_yb**2) + 0.3 * np.sqrt(mean_rg**2 + mean_yb**2)
    #make colour between 0 and 1
    # colorfulness = colorfulness / 200

    return colorfulness

3. Proportion - Devesh

In [77]:
standard_proportions = {
        'sq': 1,        # Square
        'r2': 1/1.414,  # Square root of 2
        'gr': 1/1.618,  # Golden ratio
        'r3': 1/1.732,  # Square root of 3
        'ds': 1/2       # Double square
    }

# Recursive function to parse JSON and calculate PM_object
def parse_json_and_calculate_PMobject(component, standard_proportions):
    sum_min_diff = 0
    n = 0

    def extract_proportion(bounds):
        x1, y1, x2, y2 = bounds
        width = x2 - x1
        height = y2 - y1
        return height / width if width != 0 else 0

    # Recursively process components and children
    def process_component(component):
        nonlocal sum_min_diff, n
        bounds = component.get("bounds", [])
        if bounds:
            proportion = extract_proportion(bounds)
            min_diff = min(abs(proportion - sp) for sp in standard_proportions.values())
            sum_min_diff += (1 - min_diff / 0.5)
            n += 1
        for child in component.get("children", []):
            process_component(child)

    process_component(component)
    return abs(sum_min_diff / n) if n != 0 else 0

# Calculate the PM_object for the entire JSON data structure
# PM_object = parse_json_and_calculate_PMobject(json_data, standard_proportions)
# PM_object

def calculate_PMlayout(width_layout, height_layout):
    # Standard proportions as per the provided screenshot

    # Calculate the layout proportion
    r_layout = height_layout / width_layout if width_layout != 0 else 0
    p_layout = r_layout if r_layout <= 1 else 1 / r_layout

    # Find the minimum difference between p_layout and the standard proportions
    min_diff = min(abs(p_layout - sp) for sp in standard_proportions.values())

    # Calculate PM_layout according to the formula
    PM_layout = 1 - (min_diff / 0.5)

    return abs(PM_layout)

# calculate_PMlayout(json_data["bounds"][2],json_data["bounds"][3])

def calculate_PM(json_data):
    pm_object = parse_json_and_calculate_PMobject(json_data, standard_proportions)
    pm_layout = calculate_PMlayout(json_data["bounds"][2],json_data["bounds"][3])
    return (pm_object + pm_layout) / 2

4. Symmetry - Mann

In [78]:
import os
import json
import math

def determine_quadrant(element_center_x, element_center_y, center_x, center_y):
    if element_center_x < center_x and element_center_y < center_y:
        return 'UL'  # Upper-Left
    elif element_center_x >= center_x and element_center_y < center_y:
        return 'UR'  # Upper-Right
    elif element_center_x < center_x and element_center_y >= center_y:
        return 'LL'  # Lower-Left
    elif element_center_x >= center_x and element_center_y >= center_y:
        return 'LR'  # Lower-Right

def calculate_element_properties(bounds):
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    center_x = bounds[0] + width / 2
    center_y = bounds[1] + height / 2
    return center_x, center_y, width, height

def extract_ui_elements(data, parent_bounds=None):
    elements = []
    bounds = data.get('bounds', parent_bounds)
    if 'children' not in data or not data['children']:
        return [{'class': data['class'], 'bounds': bounds}]
    for child in data['children']:
        elements.extend(extract_ui_elements(child, bounds))
    return elements

# Function to calculate the symmetry score of a single UI screen
def calculate_symmetry(data):
    screen_bounds = data['bounds']
    screen_center_x = (screen_bounds[2] + screen_bounds[0]) / 2
    screen_center_y = (screen_bounds[3] + screen_bounds[1]) / 2

    # Extract UI elements
    ui_elements = extract_ui_elements(data)

    # Organize elements by quadrant
    quadrants = {'UL': [], 'UR': [], 'LL': [], 'LR': []}
    for element in ui_elements:
        center_x, center_y, width, height = calculate_element_properties(element['bounds'])
        quadrant = determine_quadrant(center_x, center_y, screen_center_x, screen_center_y)
        quadrants[quadrant].append({
            'center_x': center_x,
            'center_y': center_y,
            'width': width,
            'height': height
        })

    # Calculate averages for each quadrant
    quadrant_sums = {key: {'x_sum': 0, 'y_sum': 0, 'width_sum': 0, 'height_sum': 0, 'count': 0}
                     for key in quadrants.keys()}

    for quadrant, elements in quadrants.items():
        for element in elements:
            quadrant_sums[quadrant]['x_sum'] += element['center_x']
            quadrant_sums[quadrant]['y_sum'] += element['center_y']
            quadrant_sums[quadrant]['width_sum'] += element['width']
            quadrant_sums[quadrant]['height_sum'] += element['height']
            quadrant_sums[quadrant]['count'] += 1

    quadrant_averages = {}
    for quadrant, sums in quadrant_sums.items():
        count = sums['count']
        if count > 0:
            quadrant_averages[quadrant] = {
                'avg_center_x': sums['x_sum'] / count,
                'avg_center_y': sums['y_sum'] / count,
                'avg_width': sums['width_sum'] / count,
                'avg_height': sums['height_sum'] / count
            }
        else:
            quadrant_averages[quadrant] = None

    # Calculate symmetry values for vertical, horizontal
    symmetry_values = {'SYM_vertical': 0, 'SYM_horizontal': 0, 'SYM_radial': 0}
    if quadrant_averages['UL'] and quadrant_averages['UR']:
        symmetry_values['SYM_vertical'] = 1 - (
            abs(quadrant_averages['UL']['avg_center_x'] - quadrant_averages['UR']['avg_center_x']) /
            screen_center_x
        )

    if quadrant_averages['UL'] and quadrant_averages['LR']:
        symmetry_values['SYM_horizontal'] = 1 - (
            abs(quadrant_averages['UL']['avg_center_y'] - quadrant_averages['LR']['avg_center_y']) /
            screen_center_y
        )

    # Since we might not have elements in all quadrants, use available symmetry values
    overall_symmetry = 0
    symmetry_count = 0
    for symmetry in symmetry_values.values():
        if symmetry > 0:
            overall_symmetry += symmetry
            symmetry_count += 1

    if symmetry_count > 0:
        overall_symmetry /= symmetry_count  # Average symmetry score

    return overall_symmetry

5. Balance - Vedika

In [79]:
import json

def screen_bounds(json_data):
    bounds = json_data['bounds']
    left, top, right, bottom = bounds
    width = right - left
    height = bottom - top
    return width, height

def parse_annotations(json_data):
    objects = []
    
    def parse_children(children):
        for item in children:
            if 'bounds' in item:
                left, top, right, bottom = item['bounds']
                width = right - left
                height = bottom - top
                objects.append({
                    'left': left,
                    'top': top,
                    'right': right,
                    'bottom': bottom,
                    'width': width,
                    'height': height
                })
            if 'children' in item:
                parse_children(item['children'])

    parse_children(json_data['children'])
    return objects

def compute_balance_scores(objects, json_data):
    screen_width, screen_height = screen_bounds(json_data)
    left_area = right_area = top_area = bottom_area = 0
    left_distance = right_distance = top_distance = bottom_distance = 0
    
    if objects == []:
        return 0
    else:
      for obj in objects:
          left_area += obj['width']*obj['height'] if (obj['left']+obj['right'])/2 < screen_width/2 else 0
          right_area += obj['width']*obj['height'] if (obj['left']+obj['right'])/2  >= screen_width/2 else 0
          top_area += obj['width']*obj['height'] if (obj['top']+obj['bottom'])/2  < screen_height/2 else 0
          bottom_area += obj['width']*obj['height'] if (obj['top']+obj['bottom'])/2 >= screen_height/2 else 0
          
          left_distance += abs((obj['left'] + obj['width'])/2 - screen_width) if (obj['left']+obj['right'])/2 < screen_width/2 else 0
          right_distance += abs((obj['left'] + obj['width'])/2 - screen_width) if (obj['left']+obj['right'])/2 < screen_width/2 else 0
          top_distance += abs((obj['top'] + obj['height'])/2 - screen_height) if (obj['top']+obj['bottom'])/2  < screen_height/2 else 0
          bottom_distance += abs((obj['top'] + obj['height'])/2 - screen_height) if (obj['top']+obj['bottom'])/2  < screen_height/2 else 0
      
      left_weight = left_area / max(left_area, right_area)    
      right_weight = right_area / max(left_area, right_area)
      top_weight = top_area / max(top_area, bottom_area)
      bottom_weight = bottom_area / max(top_area, bottom_area)
      
      vertical_balance = abs(left_weight - right_weight)
      horizontal_balance = abs(top_weight - bottom_weight)
      
      balance_measure = 1 - (vertical_balance + horizontal_balance) / 2
    
    return balance_measure

def balance_score(json_data):
    objects = parse_annotations(json_data)
    balance_score = compute_balance_scores(objects, json_data)
    # print("Balance Score:", balance_score)
    return balance_score

**Calculate Everything**

In [80]:
# import zipfile
# import os

# # Assuming we have a path to the zip file and a target directory
# zip_file_path = './ui_layout_vectors.zip'  # Replace with your zip file path
# target_directory = './'  # Replace with your target directory

# # Create target directory if it does not exist
# if not os.path.exists(target_directory):
#     os.makedirs(target_directory)

# # Extract the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(target_directory)

# # The code above assumes the paths are known and correctly provided.
# # If you have the zip file already in your environment, you can adjust the paths accordingly.


In [85]:
output_csv = './measures.csv'
json_folder = './semantic_annotations'
image_folder = './combined'

page_names = []
balance_measures = []
colour_measures = []
symmetry_measures = []
proportion_measures = []
density_measures = []

progress_bar = tqdm(total=len(os.listdir(json_folder)), desc="Processing JSON files")

for filename in os.listdir(json_folder)[:500]:
    if filename.endswith('.json'):
        progress_bar.update(1)  # Update progress bar
        page_number = filename.split('.')[0]
        page_names.append(page_number)

        with open(os.path.join(json_folder, filename), 'r', encoding='utf-8') as f:
            data = json.load(f)

        balance = balance_score(data)
        balance_measures.append(balance)

        proportion = calculate_PM(data)
        proportion_measures.append(proportion)

        symmetry = calculate_symmetry(data)
        symmetry_measures.append(symmetry)

        density = calculate_density(data)
        density_measures.append(density)

        # Load corresponding image for colour calculation
        image_filename = page_number + '.jpg'
        image_path = os.path.join(image_folder, image_filename)
        print(image_path)
        if os.path.exists(image_path):
            colour = calculate_colorfulness(image_path)
            colour_measures.append(colour)
            print(f"Color value for {page_number} : {colour}")
        else:
            # print("no file exists")
            colour_measures.append(None)  # Handle case where image is not found
        print("------")

        # Log progress every 1000 pages
        if len(page_names) % 2000 == 0:
            print(f"Processed {len(page_names)} pages")

progress_bar.close()

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Page': page_names,
    'Balance': balance_measures,
    'Colour': colour_measures,
    'Symmetry': symmetry_measures,
    'Proportion': proportion_measures,
    'Density': density_measures
})

# Calculate final score (average of all measures)
results_df['Colour'] = results_df['Colour'] / 200 # to scale between 0-1
results_df['Final Score'] = results_df[['Balance', 'Colour', 'Symmetry', 'Proportion', 'Density']].mean(axis=1)
# results_df['Final Score'] = results_df[['Balance', 'Symmetry', 'Proportion', 'Density']].mean(axis=1)

# Filtering outliers (dropping 1 out of 300 entries, negligible)
results_df = results_df[results_df['Final Score'] <= 1] 

# Save results to CSV
results_df.to_csv(output_csv, index=False)

print("Results saved to", output_csv)

Processing JSON files:   0%|          | 14/132524 [00:00<32:36, 67.71it/s]

./combined/57896.jpg
Color value for 57896 : 113.76918282175397
------
./combined/66332.jpg
Color value for 66332 : 109.03500549843314
------
./combined/30552.jpg
Color value for 30552 : 102.96049425592253
------
./combined/37297.jpg
Color value for 37297 : 79.69859957943038
------
./combined/3721.jpg
Color value for 3721 : 110.61703074856727
------
./combined/51041.jpg
Color value for 51041 : 106.55908898826954
------
./combined/2833.jpg
Color value for 2833 : 99.70287120267602
------
./combined/29289.jpg
Color value for 29289 : 141.2665910474117
------
./combined/56784.jpg
Color value for 56784 : 139.55531551871402
------
./combined/49888.jpg
Color value for 49888 : 124.20562093750846
------
./combined/23684.jpg
Color value for 23684 : 100.38768179801055
------
./combined/24141.jpg
Color value for 24141 : 98.7516376100094
------
./combined/59471.jpg
Color value for 59471 : 107.12574870643724
------
./combined/42397.jpg
Color value for 42397 : 77.40968233135798
------
./combined/13232

Processing JSON files:   0%|          | 31/132524 [00:00<30:23, 72.65it/s]

Color value for 22996 : 125.47562231742675
------
./combined/38162.jpg
Color value for 38162 : 26.09949902589382
------
./combined/47195.jpg
Color value for 47195 : 83.1495926369005
------
./combined/16030.jpg
Color value for 16030 : 82.6797096849617
------
./combined/40650.jpg
Color value for 40650 : 123.82279885848212
------
./combined/41942.jpg
Color value for 41942 : 155.4382334265938
------
./combined/26486.jpg
Color value for 26486 : 100.53424314812015
------
./combined/21343.jpg
Color value for 21343 : 117.08262431714704
------
./combined/54243.jpg
Color value for 54243 : 85.88612486185934
------
./combined/29773.jpg
Color value for 29773 : 145.25853405475428
------
./combined/34842.jpg
Color value for 34842 : 184.38219508385583
------
./combined/63130.jpg
Color value for 63130 : 167.56197521563928
------
./combined/48260.jpg
Color value for 48260 : 124.88582120884865
------
./combined/28861.jpg
Color value for 28861 : 117.74802363476559
------
./combined/32095.jpg
Color value f

Processing JSON files:   0%|          | 47/132524 [00:00<30:29, 72.42it/s]

...

Color value for 18868 : 118.00679596706706
------
./combined/2976.jpg
Color value for 2976 : 183.6014736236187
------
./combined/59534.jpg
Color value for 59534 : 156.51081918631132
------
./combined/24004.jpg
Color value for 24004 : 89.04008381195911
------
./combined/26039.jpg
Color value for 26039 : 172.3146139696479
------
./combined/70659.jpg
Color value for 70659 : 98.94208549725406
------
./combined/9269.jpg
Color value for 9269 : 153.531987261877
------
./combined/45517.jpg
Color value for 45517 : 112.03967754120592
------
./combined/396.jpg
Color value for 396 : 125.09536374277968
------
./combined/69182.jpg
Color value for 69182 : 147.88991509257232
------
./combined/21656.jpg
Color value for 21656 : 165.1163584221186
------
./combined/26193.jpg
Color value for 26193 : 172.27568322612035
------
./combined/16525.jpg
Color value for 16525 : 180.5512752332203
------
./combined/40345.jpg
Color value for 40345 : 157.7657471861137
------
./combined/22979.jpg
Color value for 22979 :

Processing JSON files:   0%|          | 244/132524 [00:03<28:37, 77.02it/s]

Color value for 20944 : 118.15557830732249
------
./combined/69028.jpg
Color value for 69028 : 22.1437971771329
------
./combined/14518.jpg
Color value for 14518 : 105.65082162913215
------
./combined/42378.jpg
Color value for 42378 : 116.6927214518173
------
./combined/32580.jpg
Color value for 32580 : 166.71733249922738
------
./combined/6036.jpg
Color value for 6036 : 88.10059311292814
------
./combined/61418.jpg
Color value for 61418 : 142.70863472302605
------
./combined/37278.jpg
Color value for 37278 : 181.8130121411703
------
./combined/63425.jpg
Color value for 63425 : 123.90225438741825
------
./combined/35245.jpg
Color value for 35245 : 177.1797863198222
------
./combined/48775.jpg
Color value for 48775 : 126.4067168130283
------
./combined/49867.jpg
Color value for 49867 : 65.12687318890448
------
./combined/53093.jpg
Color value for 53093 : 106.16270834708195
------
./combined/54756.jpg
Color value for 54756 : 165.44099950210554
------
./combined/29266.jpg
Color value for 

Processing JSON files:   0%|          | 257/132524 [00:03<29:56, 73.63it/s]

Color value for 37782 : 167.47463290997544
------
./combined/66627.jpg
Color value for 66627 : 114.79260083091646
------
./combined/1209.jpg
Color value for 1209 : 112.53259965087412
------
./combined/30047.jpg
Color value for 30047 : 155.1684549345802
------
./combined/13727.jpg
Color value for 13727 : 120.4766854836793
------
Results saved to ./measures.csv
